In [3]:
import os
import re
from io import StringIO
# import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS

from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader
from pypdf import PdfReader
import tempfile
import time




custom_prompt_template = """ Use the following pieces of information to answer the user's question.
If you don't know the answer, please just say that you don't know the answer, don't try to make up
an answer.

Context : {context}
Question : {question}

Only returns the helpful and reasonable answer below and nothing else.
Helpful answer:
"""
def set_custom_prompt(custom_prompt_template):
    prompt = PromptTemplate(template=custom_prompt_template, input_variables=['context',
                                                                              'question'])
    return prompt

def load_llm():
    n_gpu_layers = 32  # Change this value based on your model and your GPU VRAM pool.
    n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
    llm = LlamaCpp(
        #model_path="llama-2-7b-chat.ggmlv3.q4_0.bin",
        model_path="th-ggml-model-q4_0.bin",
        callback_manager=callback_manager,
        verbose=True,n_ctx = 4096, temperature = 0.1, max_tokens = 4096,
        n_batch=n_batch
    )
    return llm

def check_duplicate(source_list):
    res = []
    for i in source_list:
        if i not in res:
            res.append(i)
    return res

def convert_to_website_format(urls):
    convert_urls = []
    for url in urls:
        # Remove any '.html' at the end of the URL
        url = re.sub(r'\.html$', '', url)
        # Check if the URL starts with 'www.' or 'http://'
        if not re.match(r'(www\.|http://)', url):
            url = 'https://' + url
        if '/index' in url:
            url = url.split('/index')[0]
        match = re.match(r'^([^ ]+)', url)
        if match:
            url = match.group(1)
        convert_urls.append(url)
    return convert_urls

def regex_source(answer):
    pattern = r"'source': '(.*?)'"
    matchs = re.findall(pattern, str(answer))
    convert_urls = convert_to_website_format(matchs)
    res_urls = check_duplicate(source_list=convert_urls)
    res_urls = filter_similar_url(res_urls)
    return res_urls


def filter_search(db_similarity):
    filter_list = []
    top_score = db_similarity[0][1]
    for index, score in enumerate(db_similarity) :
        if score[1] - top_score <= 0.05:
              filter_list.append(score)
    return filter_list  

def load_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs = {'device': 'cpu'})
    return embeddings


def create_vector(_pages,_embeddings):
    db = FAISS.from_documents(_pages, _embeddings)
    return db

def load_docs(pdf_file):
    st.info("`Reading doc ...`")
    with tempfile.NamedTemporaryFile(delete=False) as temp_file:
        temp_path = temp_file.name
        temp_file.write(pdf_file.read())
        
    loader = PyPDFLoader(temp_path)
    docs = loader.load_and_split()
    return docs

def split_texts(text, chunk_size, overlap):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size= chunk_size, chunk_overlap = overlap)
    splits = text_splitter.split_text(text)
    return splits


def load_docs(docs_path):
    loader = DirectoryLoader(docs_path, glob="**/*.pdf")
    documents = loader.load()
    return documents

def split_docs(documents,chunk_size=1000):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    sp_docs = text_splitter.split_documents(documents)
    return sp_docs


In [6]:
sample_text = """ผู้ตัดสินแทบเหวอ แจกใบเหลืองให้นักเตะ แต่โดนนักเตะแจกการ์ดอูโน่ย้อนกลับ เพื่อล้างใบเหลือง
ขึ้นชื่อว่าแมตช์ฟุตบอลการกุศลก็ย่อมมีบรรยากาศผ่อนคลายมากกว่าตึงเครียด ดังเช่นในเกมคู่ระหว่าง Sidemen FC พบกับ YouTube all-star เมื่อวานนี้ (9 กันยายน) ได้เกิดมีมดังไปทั่วโลกโซเชี่ยลขึ้นจนแฟนบอลฮาก๊ากกันทั้งโลก
แม้จะเป็นแค่การเตะฟุตบอลเพื่อการกุศลแต่ก็มีการปะทะกันบ้างเพราะในนาทีที่ 77 ของเกม แม็กซ์ ฟอร์ช แข้งฝั่งทีม YouTube all-star ทำฟาวล์ใส่ ไซมอน มินเตอร์ แข้งฝั่งทีม Sidemen FC ก่อนที่ผู้ตัดสิน มาร์ค แคลตเทนเบิร์ก จะเข้ามาแจกใบเหลืองให้แก่ แม็กซ์ ฟอร์ช
ทันใดนั้น แม็กซ์ ฟอร์ช ก็ควักการ์ดอูโน่ย้อนกลับที่ซ่อนอยู่ออกมาชูใส่ผู้ตัดสิน แล้วทำสัญญาณชี้นิ้วบอกว่าใบเหลืองนี้เป็นโมฆะเพราะเขาหงายการ์ดย้อนกลับใส่คืนแล้ว ทำเอาผู้ตัดสินเกือบกลั้นหัวเราะไม่อยู่
ส่วนผลการแข่งขันเป็นทีม Sidemen FC ของยูทูบเบอร์ชื่อดัง KSI ชนะทีม YouTube all-star ของยูทูบเบอร์สุดเกรียน iShowSpeed ด้วยสกอร์ 8-5 ระดมทุนได้กว่า 2.4 ล้านปอนด์ (107 ล้านบาท) โดยรายได้ทั้งหมดจะนำไปบริจาคแก่องค์กรเกี่ยวกับสุขภาพจิต มะเร็ง และศูนย์พัฒนาเยาวชน ทั่วสหราชอาณาจักร
"""

In [7]:
from googletrans import Translator
translator = Translator(service_urls  = ["translate.google.com"])
print(translator.translate(text = sample_text, src = "th", dest="en").text)

The referee is almost absent.Give a yellow card to the playersBut was back by the players of the UNO cardTo wash the yellow card
Reputed to have a charity football match, there is a more relaxing atmosphere than tenseFor example, in a pair of games between Sidemen FC, meet YouTube All-Star yesterday (9 September). There was a famous social world until the world's Hokka football fans.
Although it was just a charity football kick, there were some clashing because in the 77th minute of the Max Forest game, Youtube All-Star made a foul at the SIDEMEN SIIMEN football team.FC before the referee Mark CalcotenbergWill come to give away the yellow card to Max Fore
Suddenly, Max Forer took out the hidden Ueno card, raising the referee.Then made a signal to the finger saying that this yellow card was void because he turned back the card backMade the referee almost unable to hold laughter
As for the competition, the Sidemen FC team of YouTube, the famous number, KSI won the YouTube team of YouTube

In [8]:
from langchain.schema import Document
documents = [Document(page_content=sample_text)]

In [9]:
documents

[Document(page_content='ผู้ตัดสินแทบเหวอ แจกใบเหลืองให้นักเตะ แต่โดนนักเตะแจกการ์ดอูโน่ย้อนกลับ เพื่อล้างใบเหลือง\nขึ้นชื่อว่าแมตช์ฟุตบอลการกุศลก็ย่อมมีบรรยากาศผ่อนคลายมากกว่าตึงเครียด ดังเช่นในเกมคู่ระหว่าง Sidemen FC พบกับ YouTube all-star เมื่อวานนี้ (9 กันยายน) ได้เกิดมีมดังไปทั่วโลกโซเชี่ยลขึ้นจนแฟนบอลฮาก๊ากกันทั้งโลก\nแม้จะเป็นแค่การเตะฟุตบอลเพื่อการกุศลแต่ก็มีการปะทะกันบ้างเพราะในนาทีที่ 77 ของเกม แม็กซ์ ฟอร์ช แข้งฝั่งทีม YouTube all-star ทำฟาวล์ใส่ ไซมอน มินเตอร์ แข้งฝั่งทีม Sidemen FC ก่อนที่ผู้ตัดสิน มาร์ค แคลตเทนเบิร์ก จะเข้ามาแจกใบเหลืองให้แก่ แม็กซ์ ฟอร์ช\nทันใดนั้น แม็กซ์ ฟอร์ช ก็ควักการ์ดอูโน่ย้อนกลับที่ซ่อนอยู่ออกมาชูใส่ผู้ตัดสิน แล้วทำสัญญาณชี้นิ้วบอกว่าใบเหลืองนี้เป็นโมฆะเพราะเขาหงายการ์ดย้อนกลับใส่คืนแล้ว ทำเอาผู้ตัดสินเกือบกลั้นหัวเราะไม่อยู่\nส่วนผลการแข่งขันเป็นทีม Sidemen FC ของยูทูบเบอร์ชื่อดัง KSI ชนะทีม YouTube all-star ของยูทูบเบอร์สุดเกรียน iShowSpeed ด้วยสกอร์ 8-5 ระดมทุนได้กว่า 2.4 ล้านปอนด์ (107 ล้านบาท) โดยรายได้ทั้งหมดจะนำไปบริจาคแก่องค์กรเกี่ยวกับสุขภา

In [16]:
import pdfbox
p = pdfbox.PDFBox()
p.extract_text('ThaiDoc.pdf') # ดึงออกมาเป็นไฟล์ txt ชื่อ data.txt

In [10]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("ThaiDoc.pdf")
pages = loader.load_and_split()

In [17]:
from multilingual_pdf2text.pdf2text import PDF2Text
from multilingual_pdf2text.models.document_model.document import Document
## create document for extraction with configurations
pdf_document = Document(
    document_path='ThaiDoc.pdf',
    language='tha'
    )
pdf2text = PDF2Text(document=pdf_document)
content = pdf2text.extract()
print(content)

INFO:multilingual_pdf2text.doc2img.parse_document:Parsing document from pdf to image
INFO:multilingual_pdf2text.ocr.image_to_text:Extracting text from images via OCR
INFO:multilingual_pdf2text.ocr.image_to_text:tesseract is not installed or it's not in your PATH. See README file for more information.


[]


In [18]:
import tika
tika.initVM()
from tika import parser
parsed = parser.from_file('ThaiDoc.pdf')
print(parsed["metadata"])
print(parsed["content"])

2023-09-11 18:04:56,200 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
2023-09-11 18:05:40,394 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
2023-09-11 18:05:41,802 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


{'pdf:unmappedUnicodeCharsPerPage': ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], 'pdf:PDFVersion': '1.5', 'xmp:CreatorTool': 'Mozilla Firefox 116.0.2', 'pdf:hasXFA': 'false', 'access_permission:modify_annotations': 'true', 'access_permission:can_print_degraded': 'true', 'X-TIKA:Parsed-By-Full-Set': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.pdf.PDFParser'], 'X-TIKA:content_handler': 'ToTextContentHandler', 'pdf:num3DAnnotations': '0', 'dcterms:created': '2023-09-10T14:35:35Z', 'dc:format': 'application/pdf; version=1.5', 'pdf:docinfo:creator_tool': 'Mozilla Firefox 116.0.2', 'pdf:overallPercentageUnmappedUnicodeChars': '0.0', 'access_permission:fill_in_form': 'true', 'pdf:hasCollection': 'false', 'pdf:encrypted': 'false', 'pdf:containsNonEmbeddedFont': 'false', 'Content-Length': '1275553', 'pdf:hasMarkedContent': 'false', 'Content-Type': 'application/pdf', 'pdf:producer': 'cairo 1.17.4 (https://cairographics.org)', 'pdf:totalUnmappedUnic

In [11]:
pages

[Document(page_content='ปัญญาประดิษฐ์Artificial intelligence (AI)\nกับการใช\ue041ประโยชน์ทางการแพทย์และเวชศาสตร์ฉุกเฉิน\nยุวเรศมคฐ์สิทธิชาญบัญชา\nภ าค ว ิ ช าเ วช ศ าส ต ร ์ ฉ ุ ก เฉ ิ น ค ณ ะแ พ ท ย ศ า ส ต ร ์ โร งพ ย า บ า ล รา ม าธ ิ บ ด ี ม ห า ว ิ ท ย าล ั ย ม ห ิ ด ล\nติดต\ue04cอผู\ue04dเขียน: ยุวเรศมคฐ\ue04e สิทธิชาญบัญชาemail: yuwares.sit@mahidol.ac.th\nวันรับ: 7พ.ค. 2564วันแก\ue04dไข: 21พ.ค. 2564วันตอบรับ: 1มิ.ย. 2564\nบทคัดย\ue040อ\nปัญญาประดิษฐ\ue04e(artificial intelligenceหรือAI)คือศาสตร\ue04eที่รวบรวมองค\ue04eความรู\ue04dในหลายสาขาวิชา โดยเฉพาะอย\ue04cางยิ่ง\nทางด\ue04dานวิทยาศาสตร\ue04eและวิศวกรรมศาสตร\ue04e มาพัฒนาให\ue04dเครื่องจักรหรือระบบคอมพิวเตอร\ue04eมีความชาญฉลาด สามารถคิด ค\ue091านวณ\nวิเคราะห\ue04e เรียนรู\ue04dและตัดสินใจ โดยใช\ue04dเหตุผลได\ue04dเสมือนสมองของมนุษย\ue04e และสามารถเรียนรู\ue04d พัฒนา และปรับปรุงกระบวนการ\nท\ue091างานเพื่อเพิ่มศักยภาพของปัญญาประดิษฐ\ue04eเองได\ue04dAIได\ue04dรับการพัฒนามาตั้งแต\ue04cปี พ.ศ. 2493จนถึงปัจจุบันจนสามารถน\ue091าไปใช\

In [13]:
import aspose.pdf as pdf

# Load the license
license = pdf.License()
license.set_license("Aspose.Total.lic")

# Load input PDF document
document = pdf.Document("ThaiDoc.pdf")

# Initialize TextAbsorber object
textAbsorber = pdf.text.TextAbsorber()

# Call Page.Accept() method to fetch text
document.pages.accept(textAbsorber)

# Get the extracted text string
text = textAbsorber.text

# # Create a TXT file and write the string
# text_file = open("PDFtoTXT.txt", "wt")
# n = text_file.write(text)
# text_file.close()

print("Conversion Completed Successfully")

KeyError: 'aspose.pdf'

In [ ]:
#DB_FAISS_PATH = "./vectorstores_clean_doc_gte-base_no_overlap/db_faiss"
#DB_FAISS_PATH = "/home/sira/sira_project/meta-Llama2/vectorstores_clean_doc_gte-base/db_faiss"
# uploaded_file = st.file_uploader("Choose a file", type = ["pdf"])
# if uploaded_file is not None:
#     splits = load_docs(uploaded_file)
#     #splits = split_texts(loaded_text, chunk_size=1000,overlap=0)
documents = load_docs('paper')
sp_docs = split_docs(documents)
#loader = PyPDFLoader("Transformer_paper.pdf")
#splits = loader.load_and_split()


In [ ]:
embeddings = load_embeddings()
db = create_vector(sp_docs,embeddings)
llm = load_llm()
qa_prompt = set_custom_prompt(custom_prompt_template)
memory = ConversationBufferMemory(memory_key="chat_history", 
                                return_messages=True, 
                                input_key="query", 
                                output_key="result")

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = db.as_retriever(search_kwargs = {'k':3}), 
    memory = memory,
    chain_type_kwargs = {"prompt":qa_prompt}) 

In [ ]:

query = ""       
start = time.time()
response = qa_chain({'query': query})
print(response["result"])
end = time.time()
print("Respone time:",int(end-start),"sec")